In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from math import sin, pi

In [37]:
class optimum_point():
    def __init__(self, f, max_iter,tol):
        self.f = f
        self.max_iter = max_iter
        self.tol = tol
    
    def golden_search(self,x_low, x_high):
        ea = sys.float_info.max
        R = (5**(0.5) - 1)/2.0
        x_l = x_low
        x_u = x_high
        _iter = 1
        d = R*(x_u - x_l)
        x1 = x_l + d
        x2 = x_u - d
        f1 = self.f(x1)
        f2 = self.f(x2)
        xopt = x1
        fopt = f1

        while (ea > self.tol and _iter < self.max_iter):
            d = d*R 
            if f1 > f2: # Shrink the interval from the left
                x_l = x2
                x2 = x1
                f2 = f1
                x1 = x_l + d
                f1 = self.f(x1)
            else:
                x_u = x1
                x1 = x2
                f1 = f2
                x2 = x_u - d
                f2 = self.f(x2)
                
            _iter += 1

            if f1 > f2:
                xopt = x1
                fopt = f1
            else: 
                xopt = x2
                fopt = f2

            # if (xopt == 0): continue

            if (xopt != 0):
                ea = (1 - R) * abs((x_u - x_l) / xopt) * 100

        return (xopt, _iter)
    
    def parabolic_interpolation(self,x_low, x_intermediate, x_high):
        def get_optima_estimate(x0, x1, x2, f0, f1, f2):
            try:
                numerator = f0 * (x1**2-x2**2) + f1 * (x2**2-x0**2) + f2 * (x0**2-x1**2)
                denominator = (2 * f0 * (x1 - x2)) + (2 * f1 * (x2 - x0)) + (2 * f2 * (x0 - x1))
                return numerator/denominator
            except:
                return x1
        calc_tol = sys.float_info.max
        x0 = x_low
        x1 = x_intermediate
        x2 = x_high
        _iter = 1
        f0 = self.f(x0)
        f1 = self.f(x1)
        f2 = self.f(x2)
        while (calc_tol >= self.tol and _iter < self.max_iter):
            x3 = get_optima_estimate(x0, x1, x2, f0, f1, f2)
            f3 = self.f(x3)
            if x3 > x1:
                if f3 > f1:
                    # Shrink the interval from the upper limit
                    x2 = x3
                    f2 = f3
                else:
                    # Shrink the interval from the lower limit
                    x0 = x1
                    x1 = x3
                    f0 = f1
                    f1 = f3
            else:
                if f3 > f1:
                    # Shrink interval from the lower limit
                    x0 = x3
                    f0 = f3
                else:
                    # shrink the interval from the upper limit
                    x2 = x1
                    f2 = f1
                    x1 = x3
                    f1 = f3
            _iter += 1
            calc_tol = abs(x2-x0) # This is not the absolute error
        return(x3, _iter)
    
    
    def newton(self,x,h):
        calc_tol = sys.float_info.max
        x_opt = x
        _iter = 1
        #f0 = f(x0)
        backWard  = lambda f,x,h : (f(x) - f(x-h))/h
        forWard   = lambda f,x,h : (f(x+h) - f(x))/h
        centered  = lambda f,x,h : (f(x+h) - f(x-h))/(2*h)
        sec_order = lambda f,x,h : (forWard(f,x,h)-backWard(f,x,h))/h
        first_differential = centered(self.f,x_opt,h)
        second_differential = sec_order(self.f,x_opt,h)
        x_opt = x_opt - (first_differential/second_differential)
        while (calc_tol >= self.tol and _iter < self.max_iter):
            first_differential = centered(self.f,x_opt,h)
            second_differential = sec_order(self.f,x_opt,h)
            x_old = x_opt
            x_opt = x_opt - (first_differential/second_differential)
            _iter += 1
            calc_tol = abs(x_old-x_opt) # This is not the absolute error
        return(x_opt, _iter)
        
sine = lambda x: -sin(x)
z =  optimum_point(sine, 200,0.0000001)
    
result = z.newton(1.6*pi,0.001)
print(f"The minimum value of the function is: {result[0]} after {result[1]} iterations")
        
result = z.parabolic_interpolation(0*pi, 0.6*pi, 1*pi)
print(f"The minimum value of the function is: {result[0]} after {result[1]} iterations")
        


        
result = z.golden_search(pi, 2*pi)

print(f"The maximum value of the function is: {result[0]} after {result[1]} iterations")
print(f"f(x) = {sin(result[0])}")




The minimum value of the function is: 4.712388980384723 after 4 iterations
The minimum value of the function is: 1.5707963241258605 after 9 iterations
The maximum value of the function is: 4.712388972518575 after 42 iterations
f(x) = -1.0
